In [3]:
import pandas as pd
import numpy as np
trainingSet = pd.read_csv("./data/train.csv")
testingSet = pd.read_csv("./data/test.csv")

In [4]:
predictionSet = pd.merge(trainingSet, testingSet, left_on='Id', right_on='Id')
#print(predictionSet.columns)

predictionSet = predictionSet.drop(columns=['Score_x'])
predictionSet = predictionSet.rename(columns={'Score_y': 'Score'})

#print(predictionSet.columns)
# predictionSet.to_csv("./data/prediction.csv", index=False)

X_train = trainingSet[trainingSet['Score'].notnull()]
df = X_train.drop(columns=['HelpfulnessNumerator', 'HelpfulnessDenominator', 'ProductId', 'UserId', 'Time', 'Summary'])
df=df.dropna(axis=0)
#print(df.isnull().sum())

In [5]:
TestSize = 25000
df = df[:TestSize]

In [6]:
from collections import defaultdict
from gensim import corpora
import re

regex = re.compile('[^a-zA-Z]')
documents = []
points = df.Text.values
for i,s in enumerate(points):
    documents.append(s)
                     
stoplist = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

texts = []
for i in range(len(documents)):
    if i%10000 == 0:
        print(i)
    try:
        texts.append([regex.sub('', word) for word in documents[i].lower().split() if word not in stoplist])
    except:
        print("error", i)

print("done")
#remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

print("here")
texts = [
    [token for token in text if frequency[token] > 10]
    for text in texts
]
print("done2")

dictionary = corpora.Dictionary(texts)
print("donzo")
corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary)

0
10000
20000
done
here
done2
donzo
Dictionary(15802 unique tokens: ['average', 'better', 'carol', 'change', 'character']...)


In [10]:
from gensim import models
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier



for topic in range(2, 10):
    print(topic)
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=topic)
    corpus_lsi = lsi_model[corpus_tfidf]

    lsi_arr = []
    for i, doc in enumerate(corpus_lsi):
        if i % 10000 == 0:
            print(i)
        try:
            temp = [i]
            for j, s in enumerate(doc):
                temp.append(s[1])
            lsi_arr.append(temp)
        except:
            lsi_arr.append([i, 0, 0, 0, 0, 0])
    
    columns = ["Id"]
    for i in range(topic):
        columns.append(str(i))
    temp = pd.DataFrame(lsi_arr, columns=columns)
    vectorMerged = pd.merge(temp, df, left_on="Id", right_on="Id")

    X_train = vectorMerged
    y_train = X_train['Score']
    X_train = X_train.fillna(value=0, axis=0)
    X = X_train.drop(columns=['Id', 'Text', 'Score'])

    runs = 5
    params = []
    for neigh in range(2, 10):
        if neigh % 10 == 0:
            print(neigh)
        model = KNeighborsClassifier(n_neighbors=neigh, algorithm='brute').fit(X, y_train)

        scores = []
        for i in range(runs):
            df_sample = X_train.sample(1000)
            sample_X = df_sample.drop(columns=['Id', 'Text', 'Score'])
            df_sample["Score"] = model.predict(sample_X)
            #df_sample

            from sklearn.metrics import mean_squared_error
            final = X_train.merge(df_sample, left_on="Id", right_on="Id")
                #.drop(columns=['Id', '1_x', '2_x', 'ProductId_x', 'UserId_x', 'HelpfulnessNumerator_x', 'HelpfulnessDenominator_x', 'Time_x', 'Summary_x', 'Text_x',
                #                                                               '1_y', '2_y', 'ProductId_y', 'UserId_y', 'HelpfulnessNumerator_y', 'HelpfulnessDenominator_y', 'Time_y', 'Summary_y', 'Text_y'])

            scores.append(mean_squared_error(final["Score_x"].to_numpy(), final["Score_y"].to_numpy()))
        params.append(np.mean(scores))

    print([params, range(1,10)])


2
0
10000
20000
[[1.2491999999999999, 1.6562000000000001, 1.6496, 1.6216000000000002, 1.6184, 1.6884000000000001, 1.6834, 1.6892], range(1, 10)]
3
0
10000
20000
[[1.2353999999999998, 1.688, 1.6088, 1.4972, 1.5564, 1.5928, 1.7812000000000001, 1.6431999999999998], range(1, 10)]
4
0
10000
20000
[[1.2518, 1.6486, 1.5547999999999997, 1.5062000000000002, 1.6585999999999999, 1.6675999999999997, 1.6182000000000003, 1.6656000000000002], range(1, 10)]
5
0
10000
20000
[[1.2630000000000001, 1.6082, 1.5976, 1.54, 1.6448, 1.6168, 1.6216000000000002, 1.8122], range(1, 10)]
6
0
10000
20000
[[1.3084, 1.669, 1.5328, 1.5558, 1.6402, 1.5916, 1.7064, 1.6862000000000001], range(1, 10)]
7
0
10000
20000
[[1.2464, 1.702, 1.559, 1.5577999999999999, 1.6235999999999997, 1.7600000000000002, 1.7508000000000004, 1.7114], range(1, 10)]
8
0
10000
20000
[[1.2944, 1.6324, 1.6782, 1.5370000000000001, 1.5714, 1.6199999999999999, 1.6987999999999999, 1.7212], range(1, 10)]
9
0
10000
20000
[[1.2608000000000001, 1.6052, 1.551

0
10000
20000


,Id,1,2,3,4,5,Score,Text
0,0,0.138157,-0.051685,-0.017310,0.059110,-0.079065,4.0,This is a charming version of the classic Dick...
1,1,0.063489,-0.075525,-0.051499,-0.020226,-0.060389,3.0,It was good but not as emotionally moving as t...
2,2,0.101273,-0.007067,-0.014868,0.021773,0.000334,3.0,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,0.090785,-0.029655,-0.003202,0.041829,-0.018472,5.0,Henry Winkler is very good in this twist on th...
4,4,0.073775,-0.040385,-0.018705,0.013831,-0.008925,4.0,This is one of the best Scrooge movies out. H...


In [51]:
lsi_arr = []
testSize = len(df)
for i,s in enumerate(points):
    if i % 10000 == 0:
        print(i)
    try:
        doc = [regex.sub('', word) for word in s.lower().split() if word not in stoplist]
        vec_bow = dictionary.doc2bow(doc)
        vec_lsi = lsi[vec_bow]
        #print(vec_lsi)
        temp = [i]
        for j, s in enumerate(vec_lsi):
            temp.append(s[1])
        lsi_arr.append(temp)
    except:
        lsi_arr.append([i, 0, 0, 0, 0, 0])

temp = pd.DataFrame(lsi_arr, columns=["Id", "1", "2", "3", "4", "5"])
vectorMerged = pd.merge(temp, df, left_on="Id", right_on="Id")

0
10000
20000


here


In [159]:
import matplotlib.pyplot as plt
%matplotlib notebook

def testClassifier(runs):
    params = []
    for neigh in range(2, 10):
        if neigh % 10 == 0:
            print(neigh)
        model = KNeighborsClassifier(n_neighbors=neigh, algorithm='brute').fit(X, y_train)

        scores = []
        for i in range(runs):
            df_sample = X_train.sample(1000)
            sample_X = df_sample.drop(columns=['Id', 'Text', 'Score'])
            df_sample["Score"] = model.predict(sample_X)
            #df_sample

            from sklearn.metrics import mean_squared_error
            final = X_train.merge(df_sample, left_on="Id", right_on="Id")
                #.drop(columns=['Id', '1_x', '2_x', 'ProductId_x', 'UserId_x', 'HelpfulnessNumerator_x', 'HelpfulnessDenominator_x', 'Time_x', 'Summary_x', 'Text_x',
                #                                                               '1_y', '2_y', 'ProductId_y', 'UserId_y', 'HelpfulnessNumerator_y', 'HelpfulnessDenominator_y', 'Time_y', 'Summary_y', 'Text_y'])

            scores.append(mean_squared_error(final["Score_x"].to_numpy(), final["Score_y"].to_numpy()))
        params.append(np.mean(scores))
    return [params, range(1,10)]

        

s = testClassifier(5)
print(s)

[[1.329, 1.7067999999999999, 1.6542000000000001, 1.5802, 1.6547999999999998, 1.6344, 1.716, 1.6474], range(1, 10)]


               1         2         3         4         5
19888  16.040354  3.651025 -0.766982 -0.765294 -0.679594
7386    1.581862  1.432338  2.172843 -1.054283  0.565987
4819    0.983748  0.783146  0.852359  0.170166 -0.206360
12659   1.163910  1.176419  2.088564 -1.476702  0.496250
16104   5.134961 -1.614096 -0.938326  0.830725 -0.522202
1    0
2    0
3    0
4    0
5    0
dtype: int64


1.322

In [95]:
X_train.head()

,Id,1,2,3,4,5,Score,Text
0,0,0.807500,0.429806,0.155306,0.591056,0.246200,4.0,This is a charming version of the classic Dick...
1,1,0.349648,0.249620,0.219579,0.317538,0.110480,3.0,It was good but not as emotionally moving as t...
2,2,1.064418,-0.511151,0.192842,0.385991,-0.054652,3.0,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,0.419592,0.216013,0.061673,0.266672,0.134686,5.0,Henry Winkler is very good in this twist on th...
4,4,0.441105,0.266155,0.099975,0.626506,0.183843,4.0,This is one of the best Scrooge movies out. H...
